In [1]:
data_path = "../data/70-15-15"
model_path = "../models"

In [2]:
!pip install tensorflow
#!pip install datasets

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd



In [14]:
treino=pd.read_csv(data_path+'/comunic_treino.csv', encoding='utf-8')
teste=pd.read_csv(data_path+'/comunic_teste.csv', encoding='utf-8')
validacao=pd.read_csv(data_path+'/comunic_valid.csv', encoding='utf-8')

In [15]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=4000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

In [16]:
def make_tfidf(coluna):
    vectorizer = TfidfVectorizer()
    vectors = np.asarray(vectorizer.fit_transform(coluna).todense())
    return vectors

In [17]:
X_train, X_test = TFIDF(treino["text"], teste["text"])
Y_train=treino.label
Y_test=teste.label
X_val=make_tfidf(validacao.text)
Y_val=validacao.label

tf-idf with 4000 features


In [19]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(treino.label), y= treino.label)
class_weights = dict(zip(np.unique(treino.label), class_weights))
class_weights

{0: 1.2253521126760563,
 1: 1.0740740740740742,
 2: 5.117647058823529,
 3: 2.4857142857142858,
 4: 0.16353383458646617,
 5: 2.1219512195121952,
 6: 4.35,
 7: 1.359375,
 8: 5.4375,
 9: 0.8529411764705882,
 10: 2.806451612903226,
 11: 4.35,
 12: 0.8613861386138614}

# Definição do modelo de classificacao DNN

In [20]:
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import optimizers

from tensorflow.python.keras.callbacks import EarlyStopping

early_stopping =  EarlyStopping(
    monitor='val_accuracy', 
    patience=4
)

def Build_Model_DNN_Text(shape, nClasses, dropout=0.4):
    model = models.Sequential()
    model.add(layers.Dense(4000,input_dim=shape,activation='relu'))
    model.add(layers.Dense(shape,activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dropout(dropout))    
    model.add(layers.Dense(1024,activation='relu'))
    model.add(layers.Dropout(dropout))    
    model.add(layers.Dense(nClasses, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    model.summary()
    return model

In [21]:
from sklearn import metrics
model_DNN = Build_Model_DNN_Text(X_train.shape[1], 13)
model_DNN.fit(X_train, Y_train,
                              validation_data=(X_test, Y_test),
                              epochs=50,
                              batch_size=128,
                              verbose=2,
                              class_weight=class_weights)
                              #callbacks=[early_stopping])
predicted = model_DNN.predict(X_test)
Y_pred = np.argmax(predicted, axis=1)
print(metrics.classification_report(Y_test, Y_pred))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 4000)              16004000  
_________________________________________________________________
dense_7 (Dense)              (None, 4000)              16004000  
_________________________________________________________________
dropout_4 (Dropout)          (None, 4000)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              4097024   
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               131200    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)              

2023-03-07 00:15:50.280665: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


9/9 - 1s - loss: 2.5757 - accuracy: 0.0522 - val_loss: 2.5632 - val_accuracy: 0.0165
Epoch 2/50
9/9 - 0s - loss: 2.5626 - accuracy: 0.0619 - val_loss: 2.5725 - val_accuracy: 0.0741
Epoch 3/50
9/9 - 0s - loss: 2.5005 - accuracy: 0.0557 - val_loss: 2.5472 - val_accuracy: 0.0576
Epoch 4/50
9/9 - 0s - loss: 2.1704 - accuracy: 0.0964 - val_loss: 2.3957 - val_accuracy: 0.1193
Epoch 5/50
9/9 - 0s - loss: 1.6266 - accuracy: 0.2246 - val_loss: 2.0812 - val_accuracy: 0.2634
Epoch 6/50
9/9 - 0s - loss: 1.1154 - accuracy: 0.5606 - val_loss: 1.8663 - val_accuracy: 0.7119
Epoch 7/50
9/9 - 0s - loss: 0.7071 - accuracy: 0.8585 - val_loss: 1.2847 - val_accuracy: 0.7160
Epoch 8/50
9/9 - 0s - loss: 0.3592 - accuracy: 0.8576 - val_loss: 1.2766 - val_accuracy: 0.7531
Epoch 9/50
9/9 - 0s - loss: 0.4548 - accuracy: 0.9567 - val_loss: 1.3359 - val_accuracy: 0.7778
Epoch 10/50
9/9 - 0s - loss: 0.1478 - accuracy: 0.9567 - val_loss: 1.1821 - val_accuracy: 0.7613
Epoch 11/50
9/9 - 0s - loss: 0.1011 - accuracy: 0.

/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Métricas

In [22]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, hamming_loss
precision = precision_score(Y_test, Y_pred,average='micro')
recall = recall_score(Y_test, Y_pred,average='micro')
f1_micro_average = f1_score(y_true=Y_test, y_pred=Y_pred, average='micro')
#roc_auc = roc_auc_score(Y_test, Y_pred, multi_class ='ovr', average = 'micro')
accuracy = accuracy_score(Y_test, Y_pred)
hloss = hamming_loss(Y_test, Y_pred)

In [23]:
print("Acurácia:",accuracy)
print("Precisão:",precision)
print("Recall:",recall)
print("F1:",f1_micro_average)
#print("Roc_auc:",roc_auc)
print("Hamming Loss:",hloss)

Acurácia: 0.7818930041152263
Precisão: 0.7818930041152263
Recall: 0.7818930041152263
F1: 0.7818930041152263
Hamming Loss: 0.21810699588477367
